In [1]:
!pip install selenium

In [2]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime, timedelta
import locale
import time
from pprint import pprint
from pymongo import MongoClient

In [3]:
locale.setlocale(locale.LC_TIME, "")

'Russian_Russia.1251'

In [4]:
client = MongoClient('localhost', 27017)
db = client['mails_database']
mails_coll = db.mails

In [5]:
driver = webdriver.Chrome()
driver.implicitly_wait(10)
driver.maximize_window()
driver.get('https://account.mail.ru/login')

In [6]:
elem_login = driver.find_element(By.NAME, 'username')
elem_login.send_keys('study.ai_172@mail.ru')
elem_login.send_keys(Keys.ENTER)

In [7]:
#elem_button = driver.find_element(By.XPATH, './/button[@data-test-id="next-button"]')
#elem_login.send_keys(Keys.ENTER)

In [8]:
elem_pass = driver.find_element(By.NAME, 'password')
elem_pass.send_keys('NextPassword172#')

In [9]:
elem_pass.send_keys(Keys.ENTER)

In [10]:
mails = driver.find_elements(By.XPATH, '//a[contains(@href, "/inbox/")]')
len(mails)

1

In [11]:
class Storage():
    def __init__(self):
        self.href_list = []
    
    def all_mails(self, mails):
        for mail in mails:
            href=mail.get_attribute('href')
            if href=='https://e.mail.ru/inbox/':
                continue
            self.href_list.append(href)
            
        self.href_list = list(set(self.href_list))
        
        return len(self.href_list)

In [12]:
a_mails = Storage()
cont = driver.find_element(By.XPATH, '//div[@aria-label="grid"]')
actions = ActionChains(driver)
actions.move_to_element(cont)
tek = 0
mails = driver.find_elements(By.XPATH, '//a[contains(@href, "/inbox/")]')
now = a_mails.all_mails(mails)
while tek != now:
    tek = now
    actions.move_to_element(mails[-1]).key_down(Keys.PAGE_DOWN).perform()
    time.sleep(3)
    mails = driver.find_elements(By.XPATH, '//a[contains(@href, "/inbox/")]')
    now = a_mails.all_mails(mails)

In [13]:
len(a_mails.href_list)

275

In [14]:
%%time
for cur_link in a_mails.href_list:
    driver.get(cur_link)
    fish = {}
    msub = driver.find_element(By.CLASS_NAME, 'thread-subject').text
    mdate = driver.find_element(By.CLASS_NAME, 'letter__date').text
    mauth = driver.find_element(By.CLASS_NAME, 'letter-contact').get_attribute('title')
    mtext = driver.find_element(By.CLASS_NAME, 'letter-body__body').text
    
    d = datetime.today() - timedelta(days=1)
    fish['mail_sub'] = msub
    fish['mail_date'] = mdate.replace('Сегодня', datetime.today().date().strftime("%d %B")).replace('Вчера', d.date().strftime("%d %B"))
    fish['mail_author'] = mauth
    fish['mail_text'] = mtext

    mails_coll.insert_one(fish)
    
    driver.back()

Wall time: 14min 59s


In [15]:
for mail in mails_coll.find({}):
    pprint(mail)

{'_id': ObjectId('622a8b9b4602df5dad2e7916'),
 'mail_author': 'noreply@e.geekbrains.ru',
 'mail_date': '8 февраля, 1:52',
 'mail_sub': 'Минус 55% на цифровые профессии 2022',
 'mail_text': 'Скидка 55%\n'
              'Привет!\n'
              'Воспользуйтесь нашими мегаскидками на популярные '
              'образовательные программы и примите участие в розыгрыше '
              'призов:\n'
              'скидка до 55% на все программы из каталога;\n'
              'видеокурсы на сумму 146 000 рублей;\n'
              'планшет iPad.\n'
              'Мы разыграем два планшета iPad среди тех, кто успеет купить '
              'программы по акции.\n'
              'Открыть распродажу\n'
              'Успейте оставить заявку, ведь количество программ по '
              'специальным ценам ограничено\n'
              'Оставайтесь на связи\n'
              'Свяжитесь с нами\n'
              'sales@geekbrains.ru\n'
              '8-800-700-68-41\n'
              'Полезные ссылки\n'
        

              '1 января 2022 г.\n'
              'При этом Министерство опубликует на своем сайте перечень '
              'автомобилей дороже 3 млн руб., как это предписывает '
              'законодательство. Однако он будет носить информационный '
              'характер и не повлечет установление повышающего коэффициента в '
              'отношении автомобилей стоимостью от 3 млн до 10 млн руб.\n'
              'Информация Министерства промышленности и торговли РФ "Перечень '
              'легковых автомобилей средней стоимостью от 3 миллионов рублей '
              '(2022 год)"\n'
              'Появился перечень дорогостоящих автомобилей на 2022 г.\n'
              'Составлен перечень легковых автомобилей средней стоимостью от 3 '
              'млн руб. для применения в налоговом периоде 2022 г. в целях '
              'повышенного налогообложения.\n'
              'Данный перечень носит информационный характер, поскольку '
              'Минпромторг предложил повысить минимал

 'mail_date': '22 февраля, 21:15',
 'mail_sub': 'Chinese vs English UI, Markdown, Fake Agency, Black Creatives, '
             'Cascade Layers',
 'mail_text': 'Email not displaying properly? View browser version.\n'
              'FEBRUARY 22 2022\n'
              'A few UI differences between Chinese and English apps\n'
              'uxdesign.cc\n'
              'Growing up in Singapore, I am exposed to cultural influences '
              'from the West as well as the East.\n'
              'UI\n'
              'Credit: UX Collective\n'
              'Thoughts On Markdown\n'
              'smashingmagazine.com\n'
              'It’s important to look at emerging content formats that try to '
              'encompass modern needs.\n'
              'Articles\n'
              'Credit: Smashing Magazine\n'
              'The elaborate con that tricked dozens into working for a fake '
              'design agency\n'
              'bbc.co.uk\n'
              'Dozens of young people were tr

 'mail_text': 'Все акции\n'
              'Онлайн-каталог\n'
              'Все предложения >\n'
              'Купить >\n'
              'Ассортимент и стоимость товаров уточняйте в Вашем торговом '
              'центре METRO или по телефону горячей линии.\n'
              'Смотреть каталог >\n'
              'Рыба\n'
              'и морепродукты\n'
              'Сыры\n'
              'Овощи и фрукты\n'
              'Чай и кофе\n'
              'Для дома и дачи\n'
              'Мясо и птица\n'
              'Электроника\n'
              'Зоотовары\n'
              'Автотовары\n'
              '\n'
              'Каталог METRO\n'
              '17.02 - 02.03.2022\n'
              '      Горячая линия METRO\n'
              '8-800-700-10-77\n'
              'пн-пт: с 07:00 до 22:00\n'
              'сб-вс: с 08:00 до 22:00\n'
              '  Вы получили это письмо, потому что подписаны на рассылку от '
              'Metro Cash & Carry Russia на адрес STUDY.AI_172@MAIL.RU. Вы '
  

              'работников которых за 2021 г. составляла не менее 5 тыс. '
              'человек, а также их дочерние и зависимые общества могут '
              'уплатить утилизационный сбор за I-III кварталы 2022 г. до 20 '
              'декабря 2022 г.\n'
              'Предприятия отрасли, попавшие под санкции, могут уплатить '
              'утилизационный сбор за IV квартал 2021 г. и за I-III кварталы '
              '2022 г. также до 20 декабря 2022 г.\n'
              '  Информация Федерального агентства воздушного транспорта от 5 '
              'марта 2022 г. "Режим временного ограничения полетов в '
              'российские аэропорты юга и центральной части России продлен"\n'
              'Ограничение полетов в 11 российских аэропортов продлено до 14 '
              'марта.\n'
              'Режим временного ограничения полетов в аэропорты юга и '
              'центральной части России продлен до 03:45 часов по московскому '
              'времени 14 марта 2022 г. Полеты 

...

              'The imperfections of Gmail\n'
              'builtformars.com\n'
              'A user experience (UX) case study, showing what they’ve done '
              'right, and what they could improve on.\n'
              'UI UX\n'
              'Credit: Built for Mars\n'
              '  You’re receiving this email because you signed up on '
              'sidebar.io. Unsubscribe\n'
              'Sacha Greif - Kyoto, Japan'}
{'_id': ObjectId('622a8e524602df5dad2e79ed'),
 'mail_author': 'no-reply@webdesignernews.com',
 'mail_date': '17 февраля, 5:34',
 'mail_sub': '6 Creative Ideas for CSS Link Hover Effects and more...',
 'mail_text': '  Streamline The Way You Manage Your Business’s Apple Devices '
              '[ad]\n'
              'STCK.MEDIA   \n'
              '6 Creative Ideas for CSS Link Hover Effects\n'
              'CSS-TRICKS.COM    COMMENTS\n'
              'The State of JS 2021\n'
              '2021.STATEOFJS.COM    COMMENTS\n'
              'Hosting for Develo

{'_id': ObjectId('622a8fa9a0618cb69282da22'),
 'mail_author': 'noreply@e.geekbrains.ru',
 'mail_date': '18 января, 22:50',
 'mail_sub': 'Хотите выиграть бесплатное обучение в GeekBrains?',
 'mail_text': 'Хочу работу\n'
              'Как устроиться на работу без опыта?\n'
              'Мы подобрали несколько специальностей в IT, в которых легко '
              'стартовать. Они подойдут и гуманитариям, и технарям. Подробнее '
              'расскажем 19–20 января и поможем подобрать ту, которая подходит '
              'вам.\n'
              '\n'
              'Участие бесплатное! Для участников будут бонусы.\n'
              'Зарегистрироваться\n'
              'Бонусы для онлайн-участников интенсива\n'
              'Розыгрыш бесплатного обучения для тех, кто выполнит задание '
              '«Колесо компетенций».\n'
              'Гранты для всех участников, кто прошел интенсив до конца.\n'
              'Участвовать в конкурсе\n'
              'Оставайтесь на связи\n'
             

              'ГВБФ, органы власти.\n'
              'Акты утверждаются Минтрудом после рассмотрения проектов на '
              'заседании Российской трехсторонней комиссии по регулированию '
              'социально-трудовых отношений.\n'
              '  Приказ Минфина России от 29 ноября 2021 г. N 196н "Об '
              'утверждении Требований к производственным и складским '
              'помещениям, используемым для производства и оборота этилового '
              'спирта (за исключением спиртов винного, виноградного, '
              'дистиллятов винного, виноградного, плодового, коньячного, '
              'кальвадосного, фармацевтической субстанции спирта этилового '
              '(этанола)"\n'
              'Минфин утвердил требования к помещениям, используемым для '
              'производства и оборота этилового спирта.\n'
              'Установлены требования к производственным и складским '
              'помещениям, используемым для производства и оборота этилового '


              'Мода и Звезды\n'
              'Влюбленность Блума, тайное удочерение и не только: 5 '
              'малоизвестных фактов о Кейт Бланшетт\n'
              'Кино и сериалы\n'
              'Больше новостей\n'
              'Мы отправили это письмо на study.ai_172@mail.ru\n'
              'Пользовательское соглашение Отписаться\n'
              'Контакты и реклама'}
{'_id': ObjectId('622a9037a0618cb69282da50'),
 'mail_author': 'subscribe@garant.ru',
 'mail_date': '3 марта, 4:31',
 'mail_sub': 'Garant. Daily monitoring of the Federal legislation at '
             '02.03.2022',
 'mail_text': 'Garant. Daily monitoring of the Federal legislation at '
              '2.03.2022\n'
              'Order of the Ministry of Labour and Social Protection of the '
              'Russian Federation No. 26n of January 28, 2022 - approval of '
              'the standard of activities for implementation of authority in '
              'the field of employment of population to provide publ

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
driver.close()